# ISOLATION FOREST FOR ANOMALY DETECTION 


Algoritmos de Isolation Forest para detecção de Anolamia na Folha de Pagamento
Importação do Modulo de Pré-processamento

In [112]:
import sys
sys.path.append("C:/Users/joaoc/Documents/MT/pagamento_servidores/src")
import process_servants as ps
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
import pandas as pd
from janitor import clean_names
from datetime import datetime

## Função para pre-processamento dos dados
Utiliza-se funções importadas do arquivo process_servants.py, no qual foram pré-definidas algumas funções para o pré-processamento dos dados

In [6]:
def process_registration_data_rendim(file_path, reference_date_str):
    """Process registration data from Excel file"""
    
    df = (
        pd.read_excel(file_path)
        .pipe(clean_names)
        .pipe(ps.convert_rendim_to_numeric)
        .pipe(ps.process_month_column)
        )    

    #Selecionar apesas as colunas necessárias para o modelo
    df_rendim = df[['cpf_servidor', 'mes', 'rendim']]
    #Se tiver valores com o CPF duplicado para o mesmo mês, somar os valores
    df_rendim = df_rendim.groupby(['cpf_servidor', 'mes'])['rendim'].sum().reset_index()

    return df_rendim

## Função para Leitura dos Dados

In [8]:
def processed_data(brute_data_file_path, reference_date, file_name): 

    registration_df = process_registration_data_rendim(brute_data_file_path, reference_date)
    
    processed_path  = "C:/Users/joaoc/Documents/MT/pagamento_servidores/data/processed/" + file_name + ".xlsx"
    print("Processed data shapes:")
    print(f"Registration data: {registration_df.shape}")
    
    #registration_df.to_excel(processed_path, index=False)
    
    return registration_df

Importação da Base de Dados

In [13]:
reference_date = '2022-04-01'
brute_file_path = 'C:/Users/joaoc/Documents/MT/pagamento_servidores/data/raw/servidores_mes_rubricas_alimentacao.xlsx'

Limpeza e Pre-processamento dos dados para tentar identificar o caso a seguir como um Anomaly Detection
![image.png](attachment:1cf54f27-e734-4093-a92f-766002dc7ed0.png)

In [86]:
df_alimentacao = processed_data(brute_file_path, reference_date, "dados_12_meses_cadastro_e_std")

C:\Users\joaoc\anaconda3\Lib\site-packages\openpyxl\styles\stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


Processed data shapes:
Registration data: (109799, 3)


## Isolation Forest

In [144]:
def isolation_forest_alimentacao(df_alimentacao, select_month):
     # Filtrar dados do mês escolhido
    df_alimentacao_serie = df_alimentacao[df_alimentacao['mes'] == select_month].copy()
    
    # Remover a coluna 'mes'
    df_alimentacao_serie = df_alimentacao_serie.drop(columns='mes')
    
    # Definir índice e selecionar coluna
    df_alimentacao_serie = df_alimentacao_serie.set_index('cpf_servidor')[['rendim']]

    contamination = 0.5

    iso_forest = IsolationForest(
        contamination = contamination,
        random_state = 42
    )
    
    x_iso_forest = df_alimentacao_serie.values
    
    iso_forest.fit(x_iso_forest)
    
    anomalias = iso_forest.predict(x_iso_forest)
    
    df_alimentacao_serie['anomalia'] = anomalias
    
    return df_alimentacao_serie 
    

In [150]:
# Definir o mes para fazer a analise (Y, m, d=1)
selected_date = datetime(2024, 6, 1)

#Run
df_alimentacao_result = isolation_forest_alimentacao(df_alimentacao, selected_date)

df_alimentacao_result[df_alimentacao_result['anomalia'] == -1]



,rendim,anomalia
cpf_servidor,,
5295266893,1681.75,-1
6402402320,835.69,-1
18794459487,1681.81,-1
20471149420,2000.00,-1
20653492472,2318.18,-1
25577603587,2000.00,-1
32319479449,2454.54,-1
38424797604,2181.81,-1
43831524904,1681.75,-1
